In [34]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/vauxhall-pollution/__results__.html
/kaggle/input/vauxhall-pollution/pollution_df.csv
/kaggle/input/vauxhall-pollution/aqg_df.csv
/kaggle/input/vauxhall-pollution/__notebook__.ipynb
/kaggle/input/vauxhall-pollution/__output__.json
/kaggle/input/vauxhall-pollution/custom.css
/kaggle/input/vauxhall-pollution/__results___files/__results___30_2.png
/kaggle/input/vauxhall-pollution/__results___files/__results___22_2.png
/kaggle/input/vauxhall-pollution/__results___files/__results___29_2.png
/kaggle/input/vauxhall-pollution/__results___files/__results___24_2.png
/kaggle/input/vauxhall-pollution/__results___files/__results___17_2.png
/kaggle/input/tfl-cycle-share-vauxhall-20182020/filtered_bike.csv
/kaggle/input/tfl-cycle-share-vauxhall-20182020/start_vauxhall.csv
/kaggle/input/tfl-cycle-share-vauxhall-20182020/end_vauxhall.csv
/kaggle/input/tfl-cycle-share-vauxhall-20182020/within_vauxhall.csv
/kaggle/input/london-weather-data/london_weather.csv


# Prep weather data

In [8]:
weather_df = pd.read_csv('../input/london-weather-data/london_weather.csv', parse_dates=True)
weather_df.rename(columns={'date': 'StartDateTime','mean_temp': 'Temp_Avg'}, inplace=True)
weather_df[['StartDateTime']] = weather_df[['StartDateTime']].apply(pd.to_datetime, format='%Y%m%d')

weather_df = weather_df.loc[(weather_df['StartDateTime'] >= '2018-01-01') & (weather_df['StartDateTime'] <= '2020-12-31')]
weather_df = weather_df.set_index('StartDateTime')
weather_df.head()


,cloud_cover,sunshine,global_radiation,max_temp,Temp_Avg,min_temp,precipitation,pressure,snow_depth
StartDateTime,,,,,,,,,
2018-01-01,6.0,0.2,16.0,12.3,6.7,5.4,0.0,99930.0,0.0
2018-01-02,6.0,0.0,13.0,10.8,7.9,3.5,10.6,100520.0,0.0
2018-01-03,6.0,1.3,25.0,12.9,8.2,5.7,8.2,99630.0,0.0
2018-01-04,6.0,0.1,15.0,9.1,9.5,6.1,2.0,99470.0,0.0
2018-01-05,6.0,2.3,31.0,5.8,7.2,5.3,0.2,99470.0,0.0


In [14]:
pollution_df = pd.read_csv('../input/vauxhall-pollution/pollution_df.csv', parse_dates=True)
pollution_df.rename(columns={'DateTime': 'StartDateTime'}, inplace=True)
pollution_df[['StartDateTime']] = pollution_df[['StartDateTime']].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')
pollution_df

,StartDateTime,NO2,PM10
0,2018-01-01 00:00:00,58.8,45.0
1,2018-01-01 01:00:00,20.8,48.0
2,2018-01-01 02:00:00,23.1,46.0
3,2018-01-01 03:00:00,24.6,26.0
4,2018-01-01 04:00:00,29.0,24.0
...,...,...,...
26276,2020-12-30 20:00:00,75.5,54.0
26277,2020-12-30 21:00:00,74.3,35.0
26278,2020-12-30 22:00:00,64.2,23.0
26279,2020-12-30 23:00:00,66.2,40.0


In [16]:
aqf_df = pd.read_csv('../input/vauxhall-pollution/aqg_df.csv')

aqf_df

,Unnamed: 0,Pollutant,AVG_Period,AQGValue,Unit
0,0,PM10,D,45,µg/m-3
1,1,PM10,Y,15,µg/m-3
2,2,NO2,H,200,µg/m-3
3,3,NO2,D,25,µg/m-3
4,4,NO2,Y,10,µg/m-3


# Bike Data Group

In [9]:
bike_df = pd.read_csv('../input/tfl-cycle-share-vauxhall-20182020/filtered_bike.csv')
bike_df[['StartDateTime','EndDateTime']] = bike_df[['StartDateTime','EndDateTime']].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S')
station_list=[813,74,270]

bike_df['Start_Vaux'] = np.where(bike_df['StartStationID'].isin(station_list), True, False)
bike_df['End_Vaux'] = np.where(bike_df['EndStationID'].isin(station_list), True, False)
bike = bike_df.set_index('StartDateTime').filter(['RentalID', 'StartDateTime','Start_Vaux','End_Vaux', 'Duration_Seconds'], axis=1)
bike['StartDateTime'] = bike.index
bike.head()

,RentalID,Start_Vaux,End_Vaux,Duration_Seconds,StartDateTime
StartDateTime,,,,,
2018-01-01 00:19:00,72337793,True,False,840,2018-01-01 00:19:00
2018-01-01 00:20:00,72337795,True,False,780,2018-01-01 00:20:00
2018-01-01 00:22:00,72337819,True,False,1920,2018-01-01 00:22:00
2018-01-01 00:22:00,72337823,True,False,1860,2018-01-01 00:22:00
2018-01-01 00:24:00,72337867,True,False,1200,2018-01-01 00:24:00


In [43]:
bike_df.dtypes

RentalID                     int64
BikeID                       int64
StartDateTime       datetime64[ns]
StartStationID               int64
StartStation                object
EndDateTime         datetime64[ns]
EndStationID                 int64
EndStation                  object
Duration_Seconds             int64
Start_Vaux                    bool
End_Vaux                      bool
dtype: object

In [35]:
#Separated between different start/end
pollution_D_count = pollution_df.set_index('StartDateTime').groupby([pd.Grouper(freq='D')]).mean()
bike_D_count = bike.groupby([pd.Grouper(freq='D')]).count()
bike_D_mean = bike.groupby([pd.Grouper(freq='D')]).mean()
bike_D = pd.merge(bike_D_count['RentalID'],bike_D_mean['Duration_Seconds'], on=['StartDateTime'])
bike_D = pd.merge(bike_D, pollution_D_count, on=['StartDateTime'])
bike_D = pd.merge(bike_D,weather_df[['Temp_Avg']], left_on=['StartDateTime'], right_on=['StartDateTime'])
bike_D.rename(columns={'RentalID': 'RentalCount'}, inplace=True)

bike_H_count = bike.groupby([pd.Grouper(freq='h')]).count()
bike_H_mean = bike.groupby([pd.Grouper(freq='h')]).mean()
bike_H = pd.merge(bike_H_count['RentalID'],bike_H_mean['Duration_Seconds'], on=['StartDateTime'])
bike_H = pd.merge(bike_H,pollution_df, left_on=['StartDateTime'], right_on=['StartDateTime'])
bike_H.rename(columns={'RentalID': 'RentalCount'}, inplace=True)

bike_H



,StartDateTime,RentalCount,Duration_Seconds,NO2,PM10
0,2018-01-01 00:00:00,33,1192.727273,58.8,45.0
1,2018-01-01 01:00:00,21,5971.428571,20.8,48.0
2,2018-01-01 02:00:00,10,402.000000,23.1,46.0
3,2018-01-01 03:00:00,17,1111.764706,24.6,26.0
4,2018-01-01 04:00:00,4,1140.000000,29.0,24.0
...,...,...,...,...,...
26274,2020-12-30 18:00:00,11,2192.727273,73.5,74.0
26275,2020-12-30 19:00:00,7,857.142857,72.2,57.0
26276,2020-12-30 20:00:00,2,420.000000,75.5,54.0
26277,2020-12-30 21:00:00,4,870.000000,74.3,35.0


# Using Bike D

In [28]:
bike_D

,RentalCount,Duration_Seconds,NO2,PM10,Temp_Avg
StartDateTime,,,,,
2018-01-01,201,1736.119403,44.012500,33.250000,6.7
2018-01-02,207,648.115942,53.866667,54.625000,7.9
2018-01-03,305,773.704918,39.312500,45.958333,8.2
2018-01-04,255,1187.058824,53.820833,24.541667,9.5
2018-01-05,280,1626.857143,73.783333,46.708333,7.2
...,...,...,...,...,...
2020-12-26,140,1673.571429,20.866667,30.666667,4.9
2020-12-27,156,1568.076923,26.004167,39.250000,7.5
2020-12-28,109,773.944954,35.933333,21.916667,1.1


In [ ]:
"""fig,ax = plt.subplots(2,1, figsize=(20,20))

sns.lineplot(x=NO2_df['Month'], y=NO2_df['Value'], hue = 'Year',data=NO2_df, ax =ax[0],errorbar=None)
ax[0].set_title("NO2 Concentrate by Month")
ax[0].set(ylabel='Volume / µg/m-3', xlabel='Month')


sns.lineplot(x=PM10_df['Month'], y=PM10_df['Value'], hue = 'Year',data=PM10_df, ax =ax[1],errorbar=None)
ax[1].set_title("PM10 Concentrate by Month")
ax[1].set(ylabel='Volume / µg/m-3', xlabel='Month')"""

In [33]:
#Correlation betwen daily avg temp and daily average PM10
bike_D.corr()




,RentalCount,Duration_Seconds,NO2,PM10,Temp_Avg
RentalCount,1.000000,0.017287,0.091341,0.080022,0.603161
Duration_Seconds,0.017287,1.000000,-0.316167,-0.138875,0.194115
NO2,0.091341,-0.316167,1.000000,0.348612,-0.254377
PM10,0.080022,-0.138875,0.348612,1.000000,-0.184177
Temp_Avg,0.603161,0.194115,-0.254377,-0.184177,1.000000


OVERALL

#when pollution is above WHO limit, how is 
r value	Interpretation
1>r≥0.8	Strong positive linear correlation
0.8>r≥0.4	Moderate positive linear correlation
0.4>r>0	Weak positive linear correlation
r=0	No correlation

Rental Count and Temp_ Avg = 0.603 =Moderate positive linear correlation = warmer temp = more bikes
Duration and N02 = -0.316 = low negative linear = trips are shorter when No2 is higher



graph plotting gy year

# same as for the Pollution

In [52]:
bike_H

bike_H1 = bike_H.filter(['StartDateTime', 'RentalCount', 'Duration_Seconds','NO2'], axis=1)
bike_H1 = bike_H1.set_index('StartDateTime').groupby([pd.Grouper(freq='ME')]).mean()
bike_H1['StartDateTime'] = bike_H1.index
bike_H1['Month'] = bike_H1['StartDateTime'].dt.month
bike_H1['Year'] = bike_H1['StartDateTime'].dt.year
#NO2_month
bike_H1

,RentalCount,Duration_Seconds,NO2,StartDateTime,Month,Year
StartDateTime,,,,,,
2018-01-31,11.701613,807.750631,59.533468,2018-01-31,1,2018
2018-02-28,11.983631,804.037934,61.595982,2018-02-28,2,2018
2018-03-31,10.168011,1255.663349,62.243548,2018-03-31,3,2018
2018-04-30,14.376389,1051.747274,58.441806,2018-04-30,4,2018
2018-05-31,19.633065,995.078212,54.110215,2018-05-31,5,2018
2018-06-30,21.329167,995.968818,45.024861,2018-06-30,6,2018
2018-07-31,21.930108,1273.299492,51.209677,2018-07-31,7,2018
2018-08-31,19.202957,1068.014281,45.365054,2018-08-31,8,2018
2018-09-30,18.733333,978.245736,50.554444,2018-09-30,9,2018


In [64]:
bike_count_pivot = bike_H1.pivot(index='Month', columns='Year', values= 'RentalCount')

bike_count_pivot.columns.name = None
bike_count_pivot.reset_index(['Month'])
bike_count_pivot['%1819Change'] = -(((bike_count_pivot[2018]-bike_count_pivot[2019]) / bike_count_pivot[2019]) * 100).round(2)
bike_count_pivot['%1920Change'] = -(((bike_count_pivot[2019]-bike_count_pivot[2020]) / bike_count_pivot[2020]) * 100).round(2)
bike_count_pivot['%1820Change'] = -(((bike_count_pivot[2018]-bike_count_pivot[2020]) / bike_count_pivot[2020]) * 100).round(2)
bike_count_pivot

,2018,2019,2020,%1819Change,%1920Change,%1820Change
Month,,,,,,
1,11.701613,13.271505,13.686828,11.83,3.03,14.50
2,11.983631,14.188988,13.755747,15.54,-3.15,12.88
3,10.168011,14.563172,9.927419,30.18,-46.70,-2.42
4,14.376389,15.993056,9.045833,10.11,-76.80,-58.93
5,19.633065,16.533602,18.034946,-18.75,8.32,-8.86
6,21.329167,16.673611,18.565278,-27.92,10.19,-14.89
7,21.930108,19.735215,18.543011,-11.12,-6.43,-18.27
8,19.202957,17.768817,20.829301,-8.07,14.69,7.81
9,18.733333,17.995833,22.540278,-4.10,20.16,16.89


In [65]:
change1819 = bike_count_pivot['%1819Change'].mean()
change1920 = bike_count_pivot['%1920Change'].mean()
change1820 = bike_count_pivot['%1820Change'].mean()
print('18-19',change1819,'\n19-20',change1819,'\n18-20',change1820)

18-19 -0.7483333333333334 
19-20 -0.7483333333333334 
18-20 -6.642500000000001


In [66]:
len_pivot = bike_H1.pivot(index='Month', columns='Year', values= 'Duration_Seconds')

len_pivot.columns.name = None
len_pivot.reset_index(['Month'])
len_pivot['%1819Change'] = -(((len_pivot[2018]-len_pivot[2019]) / len_pivot[2019]) * 100).round(2)
len_pivot['%1920Change'] = -(((len_pivot[2019]-len_pivot[2020]) / len_pivot[2020]) * 100).round(2)
len_pivot['%1820Change'] = -(((len_pivot[2018]-len_pivot[2020]) / len_pivot[2020]) * 100).round(2)
len_pivot

,2018,2019,2020,%1819Change,%1920Change,%1820Change
Month,,,,,,
1,807.750631,804.366905,812.308526,-0.42,0.98,0.56
2,804.037934,842.132595,746.475820,4.52,-12.81,-7.71
3,1255.663349,949.844148,1086.799804,-32.20,12.60,-15.54
4,1051.747274,1075.590649,1524.887488,2.22,29.46,31.03
5,995.078212,991.167811,1607.315010,-0.39,38.33,38.09
6,995.968818,969.478143,1607.332402,-2.73,39.68,38.04
7,1273.299492,961.833518,1256.073325,-32.38,23.43,-1.37
8,1068.014281,1185.022951,1239.977988,9.87,4.43,13.87
9,978.245736,1260.227274,1126.064039,22.38,-11.91,13.13


In [68]:
change1819 = len_pivot['%1819Change'].mean()
change1920 = len_pivot['%1920Change'].mean()
change1820 = len_pivot['%1820Change'].mean()
print('18-19',change1819,'\n19-20',change1920,'\n18-20',change1820)

18-19 -6.715833333333333 
19-20 11.076666666666666 
18-20 5.6933333333333325
